In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [ ]:
from ipynb.fs.full.LIGO_data_simulation_functions import noise, wave, classify_dataset, norm

In [ ]:
fs = 4096
dt = 1/fs
data_size = 5*fs
sample_size = 10000

In [ ]:
# generate data
train_data, train_labels= classify_dataset(sample_size,data_size)
test_data, test_labels = classify_dataset(sample_size,data_size)
train_data = norm(train_data)
test_data = norm(test_data)

In [ ]:
train_class = train_labels[:,0]
train_snr = train_labels[:,1]
test_class = test_labels[:,0]
test_snr = test_labels[:,1]

In [ ]:
# save data
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/train data.npz", train_data=train_data)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/train labels.npz", train_labels=train_labels)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/test data.npz", test_data=test_data)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/test labels.npz", test_labels=test_labels)

In [ ]:
# import data 
train_data = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/train data.npz")['train_data']
train_labels = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/train labels.npz")['train_labels']
test_data = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/test data.npz")['test_data']
test_labels = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/test labels.npz")['test_labels']

In [ ]:
# CNN
model = models.Sequential([
    layers.Reshape((train_data.shape[-1],1),input_shape=[train_data.shape[-1]]),
    layers.Conv1D(8,3, activation='relu'),
    layers.MaxPooling1D(4),
    layers.Dense(8, activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
  ])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
# train CNN
history = model.fit(train_data, train_class, epochs=30, 
                    validation_data=(test_data, test_class))

In [ ]:
model.save_weights("/Users/jkliao117/Desktop/MSci/updated simulated data/class_model_weights.ckpt".format(epoch=30))

In [ ]:
model.load_weights("/Users/jkliao117/Desktop/MSci/updated simulated data/class_model_weights.ckpt")

In [ ]:
# evaluate CNN on test data
test_loss, test_acc = model.evaluate(test_data, test_class, verbose=2)
predictions = np.reshape(np.rint(model.predict(test_data)),(sample_size,))
difference = predictions-test_class

In [ ]:
binary_difference = np.rint(difference)
snr_range = np.arange(0,5,0.01)
snr_accuracy = np.zeros_like(snr_range)
signal_sample_size = np.sum(test_class != 0)
for i in range(len(snr_range)):
    mask = (test_snr > snr_range[i])
    snr_accuracy[i] = np.sum(binary_difference[mask] == 0)/np.sum(mask)    

In [ ]:
print("for GW signal with SNR greater than", snr_range[-1], ", the model gives an accuracy of", snr_accuracy[-1]*100, "%")

In [ ]:
plt.figure(figsize=(7.5,5))
plt.plot(snr_range,snr_accuracy*100, linewidth=2, color="steelblue")
plt.xlabel("Signal-to-Noise Ratio",fontsize=20)
plt.ylabel("Accuracy (%)",fontsize=20)
plt.ylim(94.5,100.5)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.locator_params(axis='x',nbins=8)
plt.locator_params(axis='y',nbins=8)
#plt.xscale('log')
#plt.yscale('log')

In [ ]:
mask0 = (test_class == 0) #negative
mask1 = (test_class == 1) #positive
TP = np.sum(predictions[mask1]==1)
FP = np.sum(predictions[mask0]==1)
TN = np.sum(predictions[mask0]==0)
FN = np.sum(predictions[mask1]==0)
false_positive = FP/(FP+TN)
false_negative = FN/(FN+TP)

In [ ]:
print(false_positive)
print(false_negative)

In [ ]:
mask_small = [(test_snr>0)&(test_snr<1)]
sens_small = np.sum(binary_difference[mask_small] == 0)/np.sum(mask_small)    
snr_range = np.arange(0,10,0.5)
sensitivity = np.zeros(len(snr_range))
for i in range(len(snr_range)):
    mask = [(test_snr>0)&(test_snr>snr_range[i]) & (test_snr<(snr_range[i]+0.5))]
    sensitivity[i] = np.sum(binary_difference[mask]==0)/np.sum(mask)

In [ ]:
plt.figure(figsize=(7.5,5))
plt.plot(snr_range+0.5,snr_range*0+100, color="r")
plt.plot(snr_range+0.5,sensitivity*100,"o") #color="steelblue")
plt.xlabel("Signal-to-Noise Ratio",fontsize=20)
plt.ylabel("Sensitivity (%)",fontsize=20)
plt.ylim(82.5,102.5)
plt.grid()
plt.xlim(0,10.5)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.locator_params(axis='x',nbins=16)
plt.locator_params(axis='y',nbins=4)